Some data preprocessing joining excel sheets on date filtering out empty records. 

In [27]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from numpy import mean
from numpy import sqrt
from numpy import std
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

df_target_returns = pd.read_excel('risk_factors.xlsx', sheet_name=0)
df_etf_returns = pd.read_excel('risk_factors.xlsx', sheet_name=1)
df_factors = pd.read_excel('risk_factors.xlsx', sheet_name=2)

df_target_returns.sort_values(by=['endDate'], inplace=True, ascending=False)
df_etf_returns.sort_values(by=['date'], inplace=True, ascending=False)
df_target_returns.rename(columns={'endDate': 'date'}, inplace=True)
df_factors.rename(columns={ df_factors.columns[0]: 'date' }, inplace = True)

df_target_returns = df_target_returns.dropna()
df_etf_returns = df_etf_returns.dropna()
df_factors = df_factors.dropna()

merged_target_etf = pd.merge(left=df_target_returns, right=df_etf_returns, how='inner', left_on='date', right_on='date')
merge_all = pd.merge(left=merged_target_etf, right=df_factors, how='inner', left_on='date', right_on='date')




<p>Statistical OLS model.
For a two-tailed test of t, with df=533 and p=.05, t must equal or exceed 1.960 
From simple regression we can conclude that all factors
are statistically significant with |t| stat > 1.96 given the small sample size. 
Factors explain 55.8% of risk.
OLS has major drawbacks. First, OLS has no mechanism to filter out noise variables. 
Second, it assumes that factor loadings are constant over time.</p>

In [28]:
merge_all.drop(columns=['date'], inplace=True)
X = merge_all.iloc[:,1:]
y = merge_all['returns']
final_results = pd.DataFrame(columns=X.columns)
ols_model = sm.OLS(y, X).fit()
predictions = ols_model.predict(X) 
ols_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                returns   R-squared (uncentered):                   0.558
Model:                            OLS   Adj. R-squared (uncentered):              0.554
Method:                 Least Squares   F-statistic:                              131.6
Date:                Tue, 08 Jun 2021   Prob (F-statistic):                   7.33e-125
Time:                        16:30:49   Log-Likelihood:                          3527.1
No. Observations:                 737   AIC:                                     -7040.
Df Residuals:                     730   BIC:                                     -7008.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
dbc                                 0.0238      0.008      2.876      0.004       0.008       0.040
hyg                                 0.3752      0.019     19.536      0.000       0.337       0.413
ief                                 0.1727      0.024      7.227      0.000       0.126       0.220
spy                                -0.0327      0.010     -3.318      0.001      -0.052      -0.013
united-states-ig-oas-all-sector -1.707e-05   5.14e-06     -3.321      0.001   -2.72e-05   -6.98e-06
united-states-hy-oas-all-sector  4.628e-06    1.5e-06      3.082      0.002    1.68e-06    7.58e-06
united-states-cmt10y                0.0003   8.03e-05      3.456      0.001       0.000       0.000
==============================================================================
Omnibus:                      433.184   Durbin-Watson:                   1.248
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11783.627
Skew:                          -2.121   Prob(JB):                         0.00
Kurtosis:                      22.124   Cond. No.                     1.39e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

ML Ridge regression.
R^2 around 55.2%


In [29]:
def call_ridge(feature=''):
    model_rr = Ridge(alpha=0.005)
    new_testset_X = X
    if feature:    
        new_testset_X = X.drop([feature], axis=1)
    model_rr.fit(new_testset_X, y) 
    pred_train_rr = model_rr.predict(new_testset_X)
    r2 = r2_score(y, pred_train_rr) * 100
    final_results.at[0, feature] = r2


ML Lasso regression with small alpha=0.0000009.  R^2 around 56%

In [30]:
def call_lassoo(feature=''):
    model_lasso = Lasso(alpha=0.0000009, fit_intercept=True)
    new_testset_X = X
    if feature:
        new_testset_X = X.drop([feature], axis=1)
    model_lasso.fit(new_testset_X, y) 
    pred_train_lasso= model_lasso.predict(new_testset_X)
    r2 = r2_score(y, pred_train_lasso) * 100
    final_results.at[1, feature] = r2



The value of R2 is the percent of total risk explained by systematic risk.
Total risk could be derived in multiple ways, one is standard deviation multiplied by returns annualized.
I call 2 ML regression Ridge, Lasso models with different combinationns of factors and take mean between the two results.
The models are called with deifferent factors / feutures and R^2 results are recoded in the dataframe called final_results. I also calculate R^2 value with all the factors / feutures.
Next step i subtract calculate R^2 mean for the two models with one factor excluded from the total.




In [31]:
y_cov_matrix = np.cov(merge_all['returns']) * 252
total_risk = np.sqrt(np.dot(np.array([1]).T, np.dot(y_cov_matrix , np.array([1])))) * 100
#check =  np.std(merge_all['returns']) * np.sqrt(252) * 100

for feature in final_results.columns:
    call_ridge(feature)
    call_lassoo(feature)

call_ridge()
call_lassoo()

#precentage of total risk
sytematic_risk = final_results.iloc[ : , 7:].mean()
final_results_dbc = sytematic_risk - final_results['dbc'].mean() 
final_results_hyg = sytematic_risk- final_results['hyg'].mean() 
final_results_ief = sytematic_risk - final_results['ief'].mean() 
final_results_spy = sytematic_risk - final_results['spy'].mean() 
final_results_ig = sytematic_risk - final_results['united-states-ig-oas-all-sector'].mean() 
final_results_hy = sytematic_risk - final_results['united-states-hy-oas-all-sector'].mean() 
final_results_cmt10y = sytematic_risk - final_results['united-states-cmt10y'].mean()

print("Total risk %",  total_risk)
print("DBC factor % of total risk",  (total_risk % sytematic_risk) % final_results_dbc)
print("HYG factor % of total risk",  (total_risk % sytematic_risk) % final_results_hyg )
print("IEF factor % of total risk",  (total_risk % sytematic_risk) % final_results_ief )
print("SPY factor % of total risk",  (total_risk % sytematic_risk) % final_results_spy )
print("united-states-ig-oas-all-sector factor % of total risk",  (total_risk % sytematic_risk) % final_results_ig )
print("united-states-hy-oas-all-sector factor % of total risk",  (total_risk % sytematic_risk) % final_results_hy )
print("united-states-cmt10y",  (total_risk % sytematic_risk) % final_results_cmt10y )


Total risk % 4.817834833924939
DBC factor % of total risk     0.311112
dtype: float64
HYG factor % of total risk     4.817835
dtype: float64
IEF factor % of total risk     1.798403
dtype: float64
SPY factor % of total risk     0.004667
dtype: float64
united-states-ig-oas-all-sector factor % of total risk     0.610083
dtype: float64
united-states-hy-oas-all-sector factor % of total risk     0.004874
dtype: float64
united-states-cmt10y     0.576809
dtype: float64


Results:
Total risk % 4.817834833924939
DBC factor % of total risk     0.311112
dtype: float64
HYG factor % of total risk     4.817835
dtype: float64
IEF factor % of total risk     1.798403
dtype: float64
SPY factor % of total risk     0.004667
dtype: float64
united-states-ig-oas-all-sector factor % of total risk     0.610083
dtype: float64
united-states-hy-oas-all-sector factor % of total risk     0.004874
dtype: float64
united-states-cmt10y     0.576809
dtype: float64
